# Resumen
- seleccionado primeras variables
- transformacion var timestamp a horas
- union de los dos dataframe

## Preguntas
- que hago con los missing_data

In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [2]:
spark = SparkSession.builder \
    .appName("SmallTestDataset") \
    .master("local[*]") \
    .getOrCreate()

In [3]:
#INPUT DATA
input_data_g = "/home/jupyter/proyect/ds_taxi_NY/green_tripdata_2024-01.parquet"
input_data_y = "/home/jupyter/proyect/ds_taxi_NY/yellow_tripdata_2024-01.parquet"

In [4]:
#READ FILE
df_g = spark.read.parquet(input_data_g)
df_y = spark.read.parquet(input_data_y)

In [5]:
#SELECCION VARIABLES GREEN DATASET
lista_vars = ['VendorID',
 'lpep_pickup_datetime',
 'lpep_dropoff_datetime',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'tip_amount',
 'total_amount']

dfg = df_g.select(lista_vars)



In [6]:
# TRANSFORMACION REDONDEO TIEMPO, Y DATE GREEN TAXI
dfg = dfg.withColumn("date_init_trip", F.date_format(F.col("lpep_pickup_datetime"), "yyyy-MM-dd"))\
         .withColumn( 'hour_init_trip',     F.hour( F.col("lpep_pickup_datetime") ))\
         .withColumn("date_arrival_trip", F.date_format(F.col("lpep_dropoff_datetime"), "yyyy-MM-dd"))\
         .withColumn( 'hour_arrival_trip',     F.hour( F.col("lpep_dropoff_datetime") )) \
         .drop(F.col('lpep_pickup_datetime'))\
         .drop( F.col('lpep_dropoff_datetime') )

In [7]:
#SELECCION VARIABLES YELLOW DATASET
lista_vars = ['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'tip_amount',
 'total_amount']

dfy = df_y.select(lista_vars)

In [8]:
# TRANSFORMACION REDONDEO TIEMPO, Y DATE YELLOW TAXI
dfy = dfy.withColumn("date_init_trip", F.date_format(F.col("tpep_pickup_datetime"), "yyyy-MM-dd"))\
         .withColumn( 'hour_init_trip',     F.hour( F.col("tpep_dropoff_datetime") ))\
         .withColumn("date_arrival_trip", F.date_format(F.col("tpep_pickup_datetime"), "yyyy-MM-dd"))\
         .withColumn( 'hour_arrival_trip',     F.hour( F.col("tpep_dropoff_datetime") )) \
         .drop(F.col('tpep_pickup_datetime'))\
         .drop( F.col('tpep_dropoff_datetime') )

In [9]:
dfy.columns == dfg.columns

True

In [10]:
dfg.columns

['VendorID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'tip_amount',
 'total_amount',
 'date_init_trip',
 'hour_init_trip',
 'date_arrival_trip',
 'hour_arrival_trip']

In [11]:
dfy.columns

['VendorID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'tip_amount',
 'total_amount',
 'date_init_trip',
 'hour_init_trip',
 'date_arrival_trip',
 'hour_arrival_trip']

In [12]:
df = dfg.union(dfy)

In [13]:
len_g = dfg.count()
len_g

56551

In [14]:
len_g = dfg.count()
dfg.select([F.count( F.when(F.col(c).isNull(),c)  ).alias(c)for c in dfg.columns]).show()

+--------+------------+------------+---------------+-------------+----------+------------+--------------+--------------+-----------------+-----------------+
|VendorID|PULocationID|DOLocationID|passenger_count|trip_distance|tip_amount|total_amount|date_init_trip|hour_init_trip|date_arrival_trip|hour_arrival_trip|
+--------+------------+------------+---------------+-------------+----------+------------+--------------+--------------+-----------------+-----------------+
|       0|           0|           0|           3415|            0|         0|           0|             0|             0|                0|                0|
+--------+------------+------------+---------------+-------------+----------+------------+--------------+--------------+-----------------+-----------------+



In [15]:
len_y = dfy.count()
len_y

2964624

In [16]:
dfy.select([F.count( F.when(F.col(c).isNull(),c)  ).alias(c) for c in dfy.columns]).show()

+--------+------------+------------+---------------+-------------+----------+------------+--------------+--------------+-----------------+-----------------+
|VendorID|PULocationID|DOLocationID|passenger_count|trip_distance|tip_amount|total_amount|date_init_trip|hour_init_trip|date_arrival_trip|hour_arrival_trip|
+--------+------------+------------+---------------+-------------+----------+------------+--------------+--------------+-----------------+-----------------+
|       0|           0|           0|         140162|            0|         0|           0|             0|             0|                0|                0|
+--------+------------+------------+---------------+-------------+----------+------------+--------------+--------------+-----------------+-----------------+



In [17]:
# test de calidad
df.count()== len_y + len_g

True

In [18]:
def sumar_missing_per_var(x):
    return x.select([F.count( F.when(F.col(c).isNull(),c)  ).alias(c) for c in x.columns]).show()

In [19]:
sumar_missing_per_var(df)

+--------+------------+------------+---------------+-------------+----------+------------+--------------+--------------+-----------------+-----------------+
|VendorID|PULocationID|DOLocationID|passenger_count|trip_distance|tip_amount|total_amount|date_init_trip|hour_init_trip|date_arrival_trip|hour_arrival_trip|
+--------+------------+------------+---------------+-------------+----------+------------+--------------+--------------+-----------------+-----------------+
|       0|           0|           0|         143577|            0|         0|           0|             0|             0|                0|                0|
+--------+------------+------------+---------------+-------------+----------+------------+--------------+--------------+-----------------+-----------------+



In [20]:
df.select([F.count( F.when(F.col(c).isNull(),c)  ).alias(c) for c in df.columns]).show()

+--------+------------+------------+---------------+-------------+----------+------------+--------------+--------------+-----------------+-----------------+
|VendorID|PULocationID|DOLocationID|passenger_count|trip_distance|tip_amount|total_amount|date_init_trip|hour_init_trip|date_arrival_trip|hour_arrival_trip|
+--------+------------+------------+---------------+-------------+----------+------------+--------------+--------------+-----------------+-----------------+
|       0|           0|           0|         143577|            0|         0|           0|             0|             0|                0|                0|
+--------+------------+------------+---------------+-------------+----------+------------+--------------+--------------+-----------------+-----------------+

